In [3]:
import matplotlib         as mpl
mpl.use('MacOSX')
import os
import errno
# import PVTIclass          as PVIT
import PVTIclass_modified as PVTImaterials
import numpy              as np

import matplotlib.pyplot  as plt
import matplotlib.cm      as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable, ImageGrid


In [4]:
# Solve the problem of plt.rc('text', usetex=True)
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2019/bin/x86_64-darwin'

In [5]:
# Memory saving
plt.close('all')

# Publication Quality Figure Parameters
plt.rc('font', family='sans-serif', serif='Arial')   # Spend a long time dealing with this. 
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('axes', labelsize=18)
plt.rc('savefig', dpi=300)


jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
jet_vals[8] = [0.0, 0, 0.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

jetcmap = plt.cm.get_cmap("YlOrRd", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
# jet_vals[8] = [0.0, 0, 0.0, 1] #change the first value 
newcmap1 = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 


In [6]:
MHD_prefix = "Bx30_isoe_asym3_big"
MHD_workdir = "/Users/yz/mount_folder_data/" + MHD_prefix + "/"
MHD_plots = MHD_workdir + "plots/"

# create a directory to save plots
if not os.path.exists(MHD_plots):
    try:
        os.makedirs(MHD_plots, 0755);print("Plots directory is created.")
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

In [7]:
# Grid size
# nxt = 400
# nyt = 300
# nzt = 900

nxt = 400
nyt = 400
nzt = 1500

In [9]:
# Units transformation
scale = 1e3  

# time
time_stop          = 50e-9
time_out_frequency = 5e-9
time_output        = int(time_stop / time_out_frequency + 1)

In [ ]:
ii = 5
time = str(int((ii) * time_out_frequency * 1e9))  # Unit in ns
#         Get data
a = PVTImaterials.PVTI(MHD_workdir, MHD_prefix, ii)  # May take a long time
a.GetData(density=True,
          Z=True,
          emission=True,
          V=True,
          B=True, 
          Ti=True, 
          Te=True,
          J=True
          )

#         Transform data shape
rho = a.cube(a.d)
Zeff = a.cube(a.Z)
Vx  = a.cube(a.V[:,0])
Vy  = a.cube(a.V[:,1])
Vz  = a.cube(a.V[:,2])
Te  = a.cube(a.Te)
Ti  = a.cube(a.Ti)
em = a.cube(a.em)
Jx  = a.cube(a.J[:,0])
Jy  = a.cube(a.J[:,1])
Jz  = a.cube(a.J[:,2])
Bx  = a.cube(a.B[:,0])
By  = a.cube(a.B[:,1])
Bz  = a.cube(a.B[:,2])

#         Set plot parameters
#         extent
bounds = a.mesh.GetBounds()
xmin   = bounds[0] * scale
xmax   = bounds[1] * scale
ymin   = bounds[2] * scale
ymax   = bounds[3] * scale
zmin   = bounds[4] * scale
zmax   = bounds[5] * scale
x_extent = [ymin-ymax/2.0, ymax-ymax/2.0, zmin, zmax]
y_extent = [xmin-xmax/2.0, xmax-xmax/2.0, zmin, zmax]

# resolution

dx = xmax/nxt/scale

#         slice position
x_point = int(nxt/2)
y_point = int(nyt/2)